# Netflix dataset without user data
During a previous hackathon, we were given a csv containing netflix data. I only had an hour and a half to produce some code. I decided to see what I could do given more time

# Imports

In [1]:
import pandas as pd
import numpy as np
import ngram
import os
from tqdm.notebook import tqdm
import zipfile
import requests

from datasets import load_dataset

In [2]:
%reload_ext line_profiler

In [3]:
# PATH HANDLING:
def get_path(relative_path: str) -> str:
    try:
        basedir = os.path.dirname(os.path.abspath(__file__)) + "\\"
    except NameError:
        basedir = os.getcwd() + "\\"
    return os.path.join(basedir, relative_path)

In [4]:
# Load actors_df
try:
    actors_df = pd.read_feather("data/actors.feather")
except:
    try:
        z = zipfile.ZipFile(get_path("data\\actors.zip"))
        actors_df = pd.read_feather(z.open("actors.feather"))
        actors.to_feather("data/actors.feather")
        try:
            del z
        except:
            pass
    except FileNotFoundError:
        print("\n1: download from https://huggingface.co/datasets/Rykari/NetflixHackathon/resolve/main/actors.zip\n2: place into data/ folder")
print(actors_df.shape)
actors_df.head(2)

(20840047, 3)


,tconst,category,primaryName
0,tt0177707,actor,William K.L. Dickson
1,tt0240586,actor,William K.L. Dickson


In [5]:
# Load directors_df
try:
    directors_df = pd.read_feather("data/directors.feather")
except:
    try:
        z = zipfile.ZipFile(get_path("data\\directors.zip"))
        directors_df = pd.read_feather(z.open("directors.feather"))
        directors_df.to_feather("data/directors.feather")
        try:
            del z
        except:
            pass
    except FileNotFoundError:
        print("\n1: download from https://huggingface.co/datasets/Rykari/NetflixHackathon/resolve/main/directors.zip\n2: place into data/ folder")
print(directors_df.shape)
directors_df.head(2)

(6130344, 4)


,tconst,category,primaryName,knownForTitles
0,tt0000001,director,William K.L. Dickson,"tt0219560,tt0308254,tt1428455,tt1496763"
1,tt0000005,director,William K.L. Dickson,"tt0219560,tt0308254,tt1428455,tt1496763"


In [6]:
# Load imdb_df
try:
    imdb_df = pd.read_feather("data/IMDB_CLEANED.feather")
except FileNotFoundError:
    z = zipfile.ZipFile(get_path("data\\IMDB_CLEANED.zip"))
    imdb_df = pd.read_feather(z.open("IMDB_CLEANED.feather"))
    imdb_df.to_feather("data/IMDB_CLEANED.feather")
    try:
        del z
    except:
        pass

print(imdb_df.shape)
imdb_df.head(2)

(879357, 9)


,tconst,titleType,title,releaseYear,runtimeMinutes,genres,imdbRating,numVotes,titleCleaned
0,tt0000009,movie,miss jerry,1894,45,Romance,5.3,204,miss jerry
1,tt0000147,movie,the corbett-fitzsimmons fight,1897,100,"Documentary,News,Sport",5.3,465,the corbett fitzsimmons fight


In [7]:
# LOAD NETFLIX DATA
cleaned_df_with_IMDB = pd.read_pickle(get_path("data\\NETFLIX_CLEANED.pickle"))
print(cleaned_df_with_IMDB.shape)
cleaned_df_with_IMDB.head(2)

(5837, 14)


,netflix_id,title,directors,cast,countries,dateAdded,releaseYear,maturityRating,duration,titleType,age_rating,suitability,genres,titleCleaned
0,81193313,chocolate,NaN,"ha ji-won, yoon kye-sang, jang seung-jo, kang ...",south korea,2019-11-30,2019,TV-14,1 Season,tvSeries,16,Young Adults,"International,Korean,Romance",chocolate
1,81197050,guatemala: heart of the mayan world,"luis ara, ignacio jaunsolo",christian morales,NaN,2019-11-30,2019,TV-G,67 min,movie,0,Kids,"Documentary,International",guatemala heart of the mayan world


In [8]:
# This function will add some additional columns so let's set them up:
cleaned_df_with_IMDB["imdb_id"] = np.NaN            # REQUIRED

cleaned_df_with_IMDB["duplicateIds"] = np.NaN       # STRING: FOR TITLES THAT COULD NOT BE FILTERED TO 1 EXACT MATCH
cleaned_df_with_IMDB["numDuplicates"] = np.int8(0)  # INT: HOW MANY OF THE SAME NAME FOUND

cleaned_df_with_IMDB["isMissing"] = np.int8(0)      # BOOL: FOR TITLES THAT COULD NOT BE FOUND AT ALL
cleaned_df_with_IMDB["isMissing"] = cleaned_df_with_IMDB["isMissing"].astype("bool")

cleaned_df_with_IMDB["ngramMatchedTitle"] = np.NaN  # STRING THAT THE NGRAM SUCCESSFULLY MATCHED AGAINST
cleaned_df_with_IMDB["ngramConfidence"] = np.NaN    # FLOAT: CONFIDENCE OF NGRAM
cleaned_df_with_IMDB["filteredOn"] = np.NaN         # STRING: WHICH STAGE DID WE FIND THE EXACT MATCH?
cleaned_df_with_IMDB["errors"] = np.NaN             # STRING: FOR DEBUGGING
print(cleaned_df_with_IMDB.shape)
cleaned_df_with_IMDB.head(2)

(5837, 22)


,netflix_id,title,directors,cast,countries,dateAdded,releaseYear,maturityRating,duration,titleType,...,genres,titleCleaned,imdb_id,duplicateIds,numDuplicates,isMissing,ngramMatchedTitle,ngramConfidence,filteredOn,errors
0,81193313,chocolate,NaN,"ha ji-won, yoon kye-sang, jang seung-jo, kang ...",south korea,2019-11-30,2019,TV-14,1 Season,tvSeries,...,"International,Korean,Romance",chocolate,NaN,NaN,0,False,NaN,NaN,NaN,NaN
1,81197050,guatemala: heart of the mayan world,"luis ara, ignacio jaunsolo",christian morales,NaN,2019-11-30,2019,TV-G,67 min,movie,...,"Documentary,International",guatemala heart of the mayan world,NaN,NaN,0,False,NaN,NaN,NaN,NaN


# Matching netflix_id to imdb_id
### This is a heavy function & a first attempt
- Might need to remove all special characters from titles - possibly replace with a space?
- How does FuzzyWuzzy perform? 
  - ANS: POORLY. 1 string can take anywhere from 25 seconds to 50 seconds to perform.
- How do ngrams perform? 
  - Can compare 5837 strings against 879357 in ~25 to 30mins
- Add multithreading / multiprocessing / .. something??
  - I seriously lack the understanding to do this


In [9]:
# We will attempt ngrams if titles without an exact match...
# So let's start with a ground truth ngram table
G = ngram.NGram(imdb_df["titleCleaned"].to_list())

In [10]:
def MatchNetflixToIMDB(DF_TO_PROCESS: pd.DataFrame, breakvar: int = 0):

    # TODO: REVISIT Function & overall logic. .explode might be faster
    def listOf_(dataframe: pd.DataFrame, column: str) -> list:
        if dataframe[~dataframe[column].isna()].shape[0]:   # If dataframe is not empty
            str_of_ = dataframe[column].values[0]           # Genres is a category dtype so need to convert to a str
            if ', ' in str(str_of_):                        # If ', ' is in the string then there's more than 1 entry
                return str_of_.split(", ")
            else:
                return [str_of_]                        # Otherwise there's a single entry
        else:
            return []                                   # incase something else breaks above, nan/string encoding issue, etc

    def isMatch(search_dataframe: pd.DataFrame) -> bool:
        if search_dataframe.shape[0] == 1:
            return True
        else:
            return False

    def isMissing(search_dataframe: pd.DataFrame) -> bool:
        if search_dataframe.shape[0] == 0:
            return True
        else:
            return False

    def add_imdbID_to_df(search_dataframe: pd.DataFrame):
        DF_TO_PROCESS.loc[i, "imdb_id"] = search_dataframe["tconst"].values[0]

    def add_missing_to_df():
        DF_TO_PROCESS.loc[i, "isMissing"] = True


    def add_duplicates_to_df(search_dataframe: pd.DataFrame):
        DF_TO_PROCESS.loc[i, "numDuplicates"] = int(search_dataframe.shape[0])
        string_of_duplicates = search_dataframe.groupby("title", as_index=False).agg({"tconst": lambda x: ', '.join(x)})["tconst"].values[0]
        DF_TO_PROCESS.loc[i, "duplicateIds"] = string_of_duplicates


    def try_ngram_match():
        title_name = current["titleCleaned"].values[0]
        try:
            title, confidence = G.search(title_name)[0]
            # title = G.find(title_name)[0] # Might be faster?
            current.loc[i, "ngramMatchedTitle"] = title
            current.loc[i, "ngramConfidence"] = confidence
            del title
            del confidence
        except IndexError: # non alphanumeric strings like 'يوم الدين' will throw an error
            current.loc[i, "errors"] = "ngram error"


    def filterBy(columnName: str) -> pd.DataFrame:
        """Compare named column value of two dataframes. Will return the compared provided it:
            - Has less reults
            - Isn't empty

            Otherwise it will return the original dataframe

        Args:
            columnName (str): Name of the column who's values you wish to compare

        Returns:
            pd.DataFrame: Filtered dataframe that isn't empty
        """

        tmp = filter[filter[columnName] == current[columnName].values[0]]
        if (tmp.shape[0] < filter.shape[0]) & (tmp.shape[0] != 0):
            return tmp
        else:
            return filter

    # break after 'breakvar' iterations - used for debugging
    if not breakvar:
        stop = DF_TO_PROCESS.shape[0]
    else:
        stop = breakvar

    missing_count = 0
    my_range = tqdm(range(0, stop)) # Check impact on performance
    for i in my_range:
        filtered = 0 # TODO: DEBUGGING VAR - Due to continue not working as intended - investigate
        current = DF_TO_PROCESS.iloc[[i]].copy()
        my_range.set_description(f"Progress: ")
        my_range.set_postfix({'Missing IMDB IDs': missing_count, "Title: ": current["title"].values[0]})

        # 1: Try to find a direct title match
        filter = imdb_df[imdb_df["titleCleaned"] == current["titleCleaned"].values[0]]

        # If .shape[0] = 1, exact match found. Add ID & go to next iteration
        if isMatch(filter):
            add_imdbID_to_df(filter)
            DF_TO_PROCESS.loc[i, "filteredOn"] = "titleCleaned"
            filtered += 1
            continue # PROCEED TO NEXT ITERATION

        # If .shape[0] = 0, exact match NOT found. try ngram match
        elif isMissing(filter):
            try_ngram_match()

            # If .shape[0] is 0, add to missing & continue to next iteration
            if current["errors"].values[0] == "ngram error": 
                filtered += 1
                missing_count += 1
                continue

            # Otherwise try again to filter
            else:
                filter = imdb_df[imdb_df["titleCleaned"] == current["ngramMatchedTitle"].values[0]]

                # If .shape[0] = 1, exact match is found. Add ID & go to next iteration
                if isMatch(filter):
                    add_imdbID_to_df(filter)
                    DF_TO_PROCESS.loc[i, "filteredOn"] = "ngramMatchedTitle"
                    filtered += 1
                    continue


        # IF CODE CONTINUES AFTER THIS POINT, ASSUME MULTIPLE ID'S FOUND (Titles with the same name)

        # 3: Try to find a single match using titleType
        if not filtered:
            filter = filterBy("titleType")
            if isMatch(filter):
                add_imdbID_to_df(filter)
                DF_TO_PROCESS.loc[i, "filteredOn"] = "titleType"
                filtered += 1
                continue

        # 4: Try to find a single match using releaseYear
        if not filtered:
            filter = filterBy("releaseYear")
            if isMatch(filter):
                add_imdbID_to_df(filter)
                DF_TO_PROCESS.loc[i, "filteredOn"] = "releaseYear"
                filtered += 1
                continue


        # 5: Try to find a single match using director
        if not filtered:
            if current["directors"].notna().values[0]:
                lst_tconst = list(filter["tconst"]) # list of remaining imdb_id's from the shortest list above
                if (len(lst_tconst) > 0) & (filtered == 0):
                    list_of_directors = listOf_(current, "directors")
                    tmp_directors = directors_df[directors_df["tconst"].isin(lst_tconst)] # Want to avoid .isin() - efficiency. Better way?
                    lst_director = list(set(tmp_directors["primaryName"]) & set(list_of_directors)) # Find matching names using set
                    # TODO: Flawed logic. Many movies have multiple directors. Use method below
                    if len(lst_director) == 1:
                        tmp_tconst = tmp_directors[tmp_directors["primaryName"]==lst_director[0]]["tconst"].values[0]
                        DF_TO_PROCESS.loc[i, "imdb_id"] = tmp_tconst
                        DF_TO_PROCESS.loc[i, "filteredOn"] = "directors"
                        filtered +=1
                        continue

        # 6: Try to find a single match using actors
        if not filtered:
            if current["cast"].notna().values[0]:
                lst_tconst = list(filter["tconst"]) # list of remaining imdb_id's
                if (len(lst_tconst) > 0) & (filtered == 0):
                    list_of_actors = listOf_(current, "cast")
                    tmp_actors = actors_df[actors_df["tconst"].isin(lst_tconst)] # Want to avoid .isin() - efficiency. Better way?
                    lst_actor = list(set(tmp_actors["primaryName"]) & set(list_of_actors)) # Get a union of matching actors between cleaned_df & tmp_actors
                    dct_tconst = {}
                    if lst_actor:
                        for actor in lst_actor: # For each actor that matches
                            try:                # create a key:value pair that counts each triggered occurence per id
                                dct_tconst[tmp_actors[tmp_actors["primaryName"] == actor]["tconst"].values[0]] += 1
                            except KeyError:
                                dct_tconst[tmp_actors[tmp_actors["primaryName"] == actor]["tconst"].values[0]] = 1
                        # TODO: Handle logic to account for rare case: multiple ID's with the same max() value
                        DF_TO_PROCESS.loc[i, "imdb_id"] = max(dct_tconst.keys()) # The ID with the highest matching count gets added
                        DF_TO_PROCESS.loc[i, "filteredOn"] = "actors"
                        filtered +=1
                        continue

        # TODO: Improve logic. Continue not working as I expect? Fix it! Spamming "& (not filtered)" is a tedious pain in the ass & affects performance
        # Added & (not filtered) in attempt to improve performance

        # If none of the above filters worked, add the duplicate id's to dataframe, set missing to true & start next iteration
        if (filter.shape[0] > 1) & (not filtered):
            add_duplicates_to_df(filter)
            missing_count += 1
            continue

        # DEBUG Function: This should never trigger. If it does then there's a flaw in the logic
        if (filter.shape[0] == 0) & (not filtered):
            add_missing_to_df()
            missing_count += 1
            continue


        # DEBUG Function: This should never trigger. If it does then there's a flaw in the logic
        if not filtered:
            DF_TO_PROCESS.loc[i, "errors"] = "not found at all"
            missing_count += 1
            continue

# For testing:
# number_of_times_to_iterate = 50
# %time MatchNetflixToIMDB(cleaned_df_with_IMDB, number_of_times_to_iterate)

%time MatchNetflixToIMDB(cleaned_df_with_IMDB)
# Took 32 mins & 50s to run through 5837 records

  0%|          | 0/5837 [00:00<?, ?it/s]

CPU times: total: 32min 36s
Wall time: 32min 50s


In [11]:
print(cleaned_df_with_IMDB.shape)
cleaned_df_with_IMDB.head(20)

(5837, 22)


,netflix_id,title,directors,cast,countries,dateAdded,releaseYear,maturityRating,duration,titleType,...,genres,titleCleaned,imdb_id,duplicateIds,numDuplicates,isMissing,ngramMatchedTitle,ngramConfidence,filteredOn,errors
0,81193313,chocolate,NaN,"ha ji-won, yoon kye-sang, jang seung-jo, kang ...",south korea,2019-11-30,2019,TV-14,1 Season,tvSeries,...,"International,Korean,Romance",chocolate,NaN,"tt11214028, tt12263402, tt18263132",3,False,NaN,NaN,NaN,NaN
1,81197050,guatemala: heart of the mayan world,"luis ara, ignacio jaunsolo",christian morales,NaN,2019-11-30,2019,TV-G,67 min,movie,...,"Documentary,International",guatemala heart of the mayan world,tt11505398,NaN,0,False,NaN,NaN,titleCleaned,NaN
2,81213894,the zoya factor,abhishek sharma,"sonam kapoor, dulquer salmaan, sanjay kapoor, ...",india,2019-11-30,2019,TV-14,135 min,movie,...,"Comedy,Drama,International",the zoya factor,tt8304386,NaN,0,False,NaN,NaN,titleCleaned,NaN
3,81082007,atlantics,mati diop,"mama sane, amadou mbow, ibrahima traore, nicol...","france, senegal, belgium",2019-11-29,2019,TV-14,106 min,movie,...,"Drama,Independent,International",atlantics,tt10199586,NaN,0,False,NaN,NaN,titleCleaned,NaN
4,80213643,chip and potato,NaN,"abigail oliver, andrea libman, briana buckmast...","canada, united kingdom",NaT,2019,TV-Y,2 Seasons,tvSeries,...,Family,chip and potato,tt9897038,NaN,0,False,NaN,NaN,titleCleaned,NaN
5,81172754,crazy people,moses inwang,"ramsey nouah, chigul, sola sobowale, ireti doy...",nigeria,2019-11-29,2018,TV-14,107 min,movie,...,"Comedy,International,Thriller",crazy people,tt8542144,NaN,0,False,NaN,NaN,releaseYear,NaN
6,81120982,i lost my body,jérémy clapin,"hakim faris, victoire du bois, patrick d'assum...",france,2019-11-29,2019,TV-MA,81 min,movie,...,"Drama,Independent,International",i lost my body,tt9806192,NaN,0,False,NaN,NaN,titleCleaned,NaN
7,81227195,kalushi: the story of solomon mahlangu,mandla dube,"thabo rametsi, thabo malema, welile nzuza, jaf...",south africa,2019-11-29,2016,TV-MA,107 min,movie,...,"Drama,International",kalushi the story of solomon mahlangu,tt3487278,NaN,0,False,NaN,NaN,titleCleaned,NaN
8,70205672,la reina del sur,NaN,"kate del castillo, cristina urgel, alberto jim...","united states, spain, colombia, mexico",NaT,2019,TV-14,2 Seasons,tvSeries,...,"Crime,International,Spanish",la reina del sur,tt1704637,NaN,0,False,NaN,NaN,titleCleaned,NaN
9,81172841,lagos real fake life,mike ezuruonye,"nonso diobi, mike ezuruonye, mercy aigbe, rex ...",NaN,2019-11-29,2018,TV-14,118 min,movie,...,"Comedy,International",lagos real fake life,tt15150376,NaN,0,False,NaN,NaN,titleCleaned,NaN


In [12]:
# Save full thing to csv:
cleaned_df_with_IMDB.to_csv("data/cleaned_df_with_IMDB.csv")

In [13]:
missing_ids = cleaned_df_with_IMDB[cleaned_df_with_IMDB["imdb_id"].isna()]
print(missing_ids.shape)
missing_ids.head(2)

(564, 22)


,netflix_id,title,directors,cast,countries,dateAdded,releaseYear,maturityRating,duration,titleType,...,genres,titleCleaned,imdb_id,duplicateIds,numDuplicates,isMissing,ngramMatchedTitle,ngramConfidence,filteredOn,errors
0,81193313,chocolate,NaN,"ha ji-won, yoon kye-sang, jang seung-jo, kang ...",south korea,2019-11-30,2019,TV-14,1 Season,tvSeries,...,"International,Korean,Romance",chocolate,NaN,"tt11214028, tt12263402, tt18263132",3,False,NaN,NaN,NaN,NaN
10,81172899,payday,cheta chukwu,"baaj adebule, ebiye victor, meg otanwa, bisola...",nigeria,2019-11-29,2018,TV-MA,110 min,movie,...,"Comedy,Independent,International",payday,NaN,"tt6250554, tt8960334",2,False,NaN,NaN,NaN,NaN


In [ ]:
# # Unfortunately some of the titleTypes are mislabled too.... GHAAAD
# # Will use this later once efficiency is improved on previous function


# imdb_df_2 = pd.read_feather("data/imdb_df_full.feather")
# imdb_df_2.rename(columns={"startYear": "releaseYear", "primaryTitle": "title", "averageRating": "imdbRating"}, inplace=True)
# imdb_df_2["titleCleaned"] = imdb_df_2[["title"]].applymap(tocleanstring)
# imdb_df_2 = imdb_df_2[
#     (imdb_df_2["titleType"] == "short") |
#     (imdb_df_2["titleType"] == "tvShort") |
#     (imdb_df_2["titleType"] == "tvMovie") |
#     (imdb_df_2["titleType"] == "tvMiniSeries") |
#     (imdb_df_2["titleType"] == "tvSpecial") |
#     (imdb_df_2["titleType"] == "video") |
#     (imdb_df_2["titleType"] == "tvPilot")
#     ].copy()
# imdb_df_2.replace('\\N', np.NaN, inplace=True)
# imdb_df_2.reset_index(drop=True, inplace=True)